# Exploratory Analysis of table `actor_name`

In [1]:
# Python lib
import os
import pandas as pd
import datetime

# External lib
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"
gv_color_seq = ['#322659','#44337A','#553C9A','#6B46C1','#805AD5','#9F7AEA','#B794F4','#D6BCFA','#E9D8FD','#FAF5FF','#E9D8FD','#D6BCFA','#B794F4','#9F7AEA','#805AD5','#6B46C1','#553C9A','#44337A','#322659']

# Local lib
import toolkit as tk

# Connect to db
tk.db_connect(os.environ.get('YELLOW_BHP'), verbose=False)

# Fetch data
actor_name = tk.db_execute('select * from bhp.actor_name')

## Filter unwanted columns

According to the wiki page, we can get rid of those columns:
- `name_type`
- `name_number`

In [2]:
actor_name.drop(columns=['name_type', 'name_number'], inplace=True)

## Table extract

In [3]:
actor_name.sample(5)

,pk_actor_name,concat_acna,is_standard_name,lang_iso,name,first_name,ordinal_text,ordinal_num,particle,title,...,creator,creation_time,modifier,modification_time,concat_name,fk_abob_name_type,begin_month,begin_day,end_month,end_day
56711,57662,AcNa57662,True,deu,Tafel,Johann Friedrich Immanuel,None,NaN,None,None,...,3.0,2014-09-11 22:47:27.880,NaN,NaT,"Tafel, Johann Friedrich Immanuel",1058.0,NaN,NaN,NaN,NaN
5316,5622,AcNa5622,True,,Cottin,"Joseph, François, Régis",None,NaN,None,None,...,11.0,2008-07-18 18:36:46.000,11.0,2013-02-14 11:00:03,"Cottin, Joseph, François, Régis",NaN,NaN,NaN,NaN,NaN
30473,31003,AcNa31003,True,,Lechère,Louis,None,NaN,None,None,...,14.0,2010-03-17 09:40:30.000,14.0,2013-02-14 11:00:03,"Lechère, Louis",NaN,NaN,NaN,NaN,NaN
55011,55959,AcNa55959,True,deu,Geoffroy,Claude-Joseph,None,NaN,None,None,...,3.0,2014-09-11 22:47:27.880,NaN,NaT,"Geoffroy, Claude-Joseph",1058.0,NaN,NaN,NaN,NaN
35282,35939,AcNa35939,True,,Herr,G,None,NaN,None,None,...,31.0,2010-05-30 12:59:15.000,31.0,2013-02-14 11:00:03,"Herr, G",NaN,NaN,NaN,NaN,NaN


## Discovery

In [4]:
tk.discover(actor_name, uniq_ex_nb=3)

Columns contain:
Total number of rows: 67293
  -      "pk_actor_name":   0.00% empty - 67293 (100.00%) uniques (eg: 49829; 49830; 49832)
  -        "concat_acna":   0.00% empty - 67293 (100.00%) uniques (eg: AcNa49829; AcNa49830; AcNa49832)
  -   "is_standard_name":   0.00% empty -     2 (  0.00%) uniques (eg: True; False)
  -        "concat_name":   0.00% empty - 63642 ( 94.57%) uniques (eg: Otte, Bern...; Staud, Joh...; Roma, Giul...)
  -      "creation_time":   0.00% empty - 40469 ( 60.14%) uniques (eg: 2013-02-20...; 2013-02-20...; 2013-02-20...)
  -           "fk_actor":   0.00% empty - 61555 ( 91.47%) uniques (eg: 46706; 46707; 46709)
  -            "creator":   0.00% empty -    89 (  0.13%) uniques (eg: 48.0; 3.0; 41.0)
  -               "name":   3.55% empty - 32301 ( 48.00%) uniques (eg: Otte; Staud; Roma)
  -           "lang_iso":   4.20% empty -    27 (  0.04%) uniques (eg: None; ita;    )
  -           "modifier":   7.31% empty -    88 (  0.13%) uniques (eg: 48.0; 3.0; 116.

## Type parsing

According to the table before, we will parse each column by the most meaningful type.

In [5]:
tk.set_types(actor_name, {
     "pk_actor_name": 'int',
       "concat_acna": 'string',
     "creation_time": 'datetime',
  "is_standard_name": 'bool',
          "fk_actor": 'int',
       "concat_name": 'string',
           "creator": 'int',
              "name": 'string',
          "lang_iso": 'string',
          "modifier": 'int',
        "first_name": 'string',
 "modification_time": 'datetime',
 "fk_abob_name_type": 'int',
             "notes": 'string',
"comment_begin_year": 'string',
  "comment_end_year": 'string',
        "apposition": 'string',
       "preposition": 'string',
          "particle": 'string',
             "title": 'string',
        "begin_year": 'int',
          "end_year": 'int',
      "ordinal_text": 'string',
       "ordinal_num": 'int',
       "begin_month": 'int',
         "begin_day": 'int',
         "end_month": 'int',
           "end_day": 'int',
})     

# So that they appear correctly
tk.set_types(actor_name, {
        "begin_year": 'string',
          "end_year": 'string',
       "begin_month": 'string',
         "end_month": 'string',
         "begin_day": 'string',
           "end_day": 'string',
})     

## Columns analysis

Here we will report the analysis of interesting information found on different columns. They are not exhaustive.

For some of the column, we will update their value.

### begin_date & end_date

We create 2 new columns, made of the joining of `begin_year`, `begin_month`, `begin_day` and `end_year`, `end_month`, `end_day`.

In [6]:
def prefix_date(date):
    if pd.isna(date): return date
    if len(str(date)) == 3: return f'0{date}'
    return date 

# Set the length of begin_year and end_year to 4
actor_name['begin_year'] = [prefix_date(d) for d in actor_name['begin_year']]
actor_name['end_year'] = [prefix_date(d) for d in actor_name['end_year']]

actor_name['begin_date'] = actor_name['begin_year'] + actor_name['begin_month'] + actor_name['begin_day']
actor_name['end_date'] = actor_name['end_year'] + actor_name['end_month'] + actor_name['end_day']
actor_name.drop(columns=['begin_year', 'begin_month', 'begin_day', 'end_year', 'end_month', 'end_day'], inplace=True)

# Parse into datetime
actor_name['begin_date'] = [datetime.datetime.strptime(d, '%Y%m%d') if pd.notna(d) else pd.NaT for d in actor_name['begin_date']]
actor_name['end_date'] = [datetime.datetime.strptime(d, '%Y%m%d') if pd.notna(d) else pd.NaT for d in actor_name['end_date']]

### creation_time 

In [7]:
# px.violin(actor_name, x='creation_time', title='Violin plot of the "creation_time" column')

### creator

In [8]:
# tk.histogram(actor_name, 'creator', 'Creator distribution', 10)

### lang_iso

Some cleaning is made on this column, in order to fit ISO639-2/T (3 letters code, native prefered, eg 'deu' instead of 'ger').

In [9]:
actor_name['lang_iso'].replace('   ', pd.NA, inplace=True)
actor_name['lang_iso'].replace('fr ', 'fra', inplace=True)
actor_name['lang_iso'].replace('Fr ', 'fra', inplace=True)
actor_name['lang_iso'].replace('FRA', 'fra', inplace=True)
actor_name['lang_iso'].replace('ang', 'eng', inplace=True)
actor_name['lang_iso'].replace('gre', 'ell', inplace=True)
actor_name['lang_iso'].replace('gal', 'gla', inplace=True)

In [10]:
# tk.histogram(actor_name, 'lang_iso', 'Language distribution', style='pie', colors=gv_color_seq)

### notes

All HTML tags, non ASCII chars and new line are removed.

In [11]:
actor_name.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
actor_name.replace({'\r\n':''}, regex=True, inplace=True)
actor_name.replace({'<p>':''}, regex=True, inplace=True)
actor_name.replace({'</p>':''}, regex=True, inplace=True)

---

In [12]:
actor_name.to_csv('../../data/actor_name.csv', index=False, sep=';', quoting=2)